In [1]:
! rm -r unilm
! git clone -b remove_torch_save https://github.com/NielsRogge/unilm.git
! cd unilm/layoutlm
! pip install unilm/layoutlm
! rm -r transformers 
! git clone https://github.com/huggingface/transformers.git 
! cd transformers 
! pip install ./transformers 
! wget https://guillaumejaume.github.io/FUNSD/dataset.zip
! unzip dataset.zip && mv dataset data && rm -rf dataset.zip __MACOSX
! sudo apt install tesseract-ocr
! pip install pytesseract

rm: cannot remove 'unilm': No such file or directory
Cloning into 'unilm'...
remote: Enumerating objects: 797, done.
remote: Total 797 (delta 0), reused 0 (delta 0), pack-reused 797
Receiving objects: 100% (797/797), 439.18 KiB | 21.96 MiB/s, done.
Resolving deltas: 100% (441/441), done.
Processing ./unilm/layoutlm
     |████████████████████████████████| 645kB 27.3MB/s 
     |████████████████████████████████| 204kB 46.3MB/s 
     |████████████████████████████████| 5.5MB 49.3MB/s 
     |████████████████████████████████| 901kB 47.9MB/s 
     |████████████████████████████████| 1.2MB 50.7MB/s 
     |████████████████████████████████| 5.6MB 47.8MB/s 
  Created wheel for layoutlm: filename=layoutlm-0.0-cp37-none-any.whl size=11486 sha256=12df254de18226a97ec428da528965c2fce175ab4eb9941d8b2fb9fa0dc8b72f
  Stored in directory: /tmp/pip-ephem-wheel-cache-xei47wvx/wheels/e8/9a/90/87de19930fb582e6176ea7912010f101efa37def32b8ced268
  Created wheel for seqeval: filename=seqeval-0.0.12-cp37-none-any.w

In [2]:
import pytesseract
import matplotlib.pyplot as plt
import numpy as np
#from sklearn.cluster import DBSCAN, OPTICS
import random
import cv2
from PIL import Image, ImageDraw, ImageFont
from glob import glob

In [3]:
! python unilm/layoutlm/examples/seq_labeling/preprocess.py --data_dir data/training_data/annotations \
                                                      --data_split train \
                                                      --output_dir data \
                                                      --model_name_or_path microsoft/layoutlm-base-uncased \
                                                      --max_len 510

! python unilm/layoutlm/examples/seq_labeling/preprocess.py --data_dir data/testing_data/annotations \
                                                      --data_split test \
                                                      --output_dir data \
                                                      --model_name_or_path microsoft/layoutlm-base-uncased \
                                                      --max_len 510

2021-05-30 07:15:17.732872: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
Downloading: 100% 606/606 [00:00<00:00, 700kB/s]
Downloading: 100% 232k/232k [00:00<00:00, 32.3MB/s]
Downloading: 100% 466k/466k [00:00<00:00, 40.9MB/s]
Downloading: 100% 112/112 [00:00<00:00, 145kB/s]
Downloading: 100% 170/170 [00:00<00:00, 232kB/s]
2021-05-30 07:15:24.242771: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [4]:
! cat data/train.txt | cut -d$'\t' -f 2 | grep -v "^$"| sort | uniq > data/labels.txt

In [8]:
from torch.nn import CrossEntropyLoss

def get_labels(path):
    with open(path, "r") as f:
        labels = f.read().splitlines()
    if "O" not in labels:
        labels = ["O"] + labels
    return labels 

labels = get_labels("data/labels.txt")
num_labels = len(labels)
label_map = {i: label for i, label in enumerate(labels)}
# Use cross entropy ignore index as padding label id so that only real label ids contribute to the loss later
pad_token_label_id = CrossEntropyLoss().ignore_index

In [9]:
from transformers import LayoutLMTokenizer
from layoutlm.data.funsd import FunsdDataset, InputFeatures
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

args = {'local_rank': -1,
        'overwrite_cache': True,
        'data_dir': 'data',
        'model_name_or_path':'microsoft/layoutlm-base-uncased',
        'max_seq_length': 512,
        'model_type': 'layoutlm',}

# class to turn the keys of a dict into attributes (thanks Stackoverflow)
class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self

args = AttrDict(args)

tokenizer = LayoutLMTokenizer.from_pretrained("microsoft/layoutlm-base-uncased")

# the LayoutLM authors already defined a specific FunsdDataset, so we are going to use this here
train_dataset = FunsdDataset(args, tokenizer, labels, pad_token_label_id, mode="train")
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset,
                              sampler=train_sampler,
                              batch_size=2)

eval_dataset = FunsdDataset(args, tokenizer, labels, pad_token_label_id, mode="test") 
eval_sampler = SequentialSampler(eval_dataset) 
eval_dataloader = DataLoader(eval_dataset, 
                             sampler=eval_sampler, 
                            batch_size=2) 

In [10]:
batch = next(iter(train_dataloader))
input_ids = batch[0][0]
tokenizer.decode(input_ids)

"[CLS] date number project ( s ) in ( s ) marketing research - cpt initiating brief 15. 03. 2000 bat korea client group client contact alberto marcheggiano kyu yeon hwang research liaison subi. jeong to be assigned by brown & williamson usa dunhill lights, cpt project name / description : market ( s ) / zone ( s ) south korea referent brand dunhill lights kent super lights mild 7 this consumer segment own brand - dunhill smokers : 150 males, 25 ~ 39 years old, abc + who live seoul / pusan competitors brands - mild7 lights smokers 150 males, 25 ~ 39 years old, abc + who live source of business - this : 250 males, ( 100 smokers 25 ~ 29 years old, 150 smokers, 30 ~ 39 years old ) abc + who are not rejectors of imported cigarette and live in seoul / pusan source of business - local premium ks smokers 250 males ( 100 smokers, 25 ~ 29 years old, 150 smokers, 30 ~ 39 years old abc + who are not rejectors of imported cigarette and live in seoul / pusan background / problem definition 1999 show

In [11]:
from transformers import LayoutLMForTokenClassification
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = LayoutLMForTokenClassification.from_pretrained("microsoft/layoutlm-base-uncased", num_labels=num_labels)
model.to(device)

Some weights of the model checkpoint at microsoft/layoutlm-base-uncased were not used when initializing LayoutLMForTokenClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing LayoutLMForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LayoutLMForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of LayoutLMForTokenClassification were not initialized from the model checkpoint at microsoft

LayoutLMForTokenClassification(
  (layoutlm): LayoutLMModel(
    (embeddings): LayoutLMEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (x_position_embeddings): Embedding(1024, 768)
      (y_position_embeddings): Embedding(1024, 768)
      (h_position_embeddings): Embedding(1024, 768)
      (w_position_embeddings): Embedding(1024, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): LayoutLMEncoder(
      (layer): ModuleList(
        (0): LayoutLMLayer(
          (attention): LayoutLMAttention(
            (self): LayoutLMSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
         

In [12]:
from transformers import AdamW
from tqdm import tqdm

optimizer = AdamW(model.parameters(), lr=5e-5)

global_step = 0
num_train_epochs = 5
t_total = len(train_dataloader) * num_train_epochs # total number of training steps 

#put the model in training mode
model.train()
for epoch in range(num_train_epochs):
  for batch in tqdm(train_dataloader, desc="Training"):
      input_ids = batch[0].to(device)
      bbox = batch[4].to(device)
      attention_mask = batch[1].to(device)
      token_type_ids = batch[2].to(device)
      labels = batch[3].to(device)

      # forward pass
      outputs = model(input_ids=input_ids, bbox=bbox, attention_mask=attention_mask, token_type_ids=token_type_ids,labels=labels)
      loss = outputs.loss

      # print loss every 100 steps
      if global_step % 100 == 0:
        print(f"Loss after {global_step} steps: {loss.item()}")

      # backward pass to get the gradients 
      loss.backward()

      #print("Gradients on classification head:")
      #print(model.classifier.weight.grad[6,:].sum())

      # update
      optimizer.step()
      optimizer.zero_grad()
      global_step += 1

Training:   1%|▏         | 1/75 [00:00<00:30,  2.46it/s]

Loss after 0 steps: 2.554445743560791


Training:  35%|███▍      | 26/75 [00:05<00:10,  4.56it/s]

Loss after 100 steps: 1.0727522373199463


Training:  68%|██████▊   | 51/75 [00:11<00:05,  4.51it/s]

Loss after 200 steps: 0.28507086634635925


Training:   1%|▏         | 1/75 [00:00<00:15,  4.65it/s]

Loss after 300 steps: 0.1907338947057724


Training: 100%|██████████| 75/75 [00:16<00:00,  4.44it/s]


In [ ]:
torch.save(model.state_dict(),'layoutlm,5.pkl')